# Tynan 548 Project File



## STEP X 
What this step does

In [718]:

import cvxpy as cp # import cvxpy

# in this problem, we will use the dynamaxsys library to import dynamical systems implemented in JAX: https://github.com/UW-CTRL/dynamaxsys
from dynamaxsys.simplecar import DynamicallyExtendedSimpleCar
from dynamaxsys.base import get_discrete_time_dynamics
from dynamaxsys.utils import linearize


import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import functools
from ipywidgets import interact


The following segment involves the rover going from the starting point to the first sampling location

In [719]:
# define the robot dynamics
wheelbase = 1.0
dt = 0.1
ct_robot_dynamics = DynamicallyExtendedSimpleCar(wheelbase=wheelbase) # robot dynamics
dt_robot_dynamics = get_discrete_time_dynamics(ct_robot_dynamics, dt=dt) # discrete time dynamics
state_dim = dt_robot_dynamics.state_dim
control_dim = dt_robot_dynamics.control_dim




In [720]:
# set up the problem parameters
planning_horizon = 15 # length of the planning horizon
num_time_steps = 50 # number of time steps to simulate
num_sqp_iterations = 10 # number of SQP iterations
t = 0. # this doesn't affect anything, but a value is needed 

# control and velocity limits
v_max = 1.5
v_min = 0.
acceleration_max = 1.0
acceleration_min = -1.0
steering_max = 0.5
steering_min = -0.5

# obstacle parameters
obstacle_location = jnp.array([1.0, 0.0]) # obstacle location
obstacle_location2 = jnp.array([3.0, -0.5]) # obstacle location
obstacle_radius = 0.5 # obstacle radius
robot_radius = 0.1 # robot radius
goal_location = jnp.array([3.5, 3.0])
goal_radius = 0.1



In [721]:
# HELPER FUNCTIONS #

# define obstacle function g(x) >= 0
# where g(x) is the distance from the obstacle
@jax.jit
def obstacle_constraint(state, obstacle, radius):
    return jnp.linalg.norm(state[:2] - obstacle[:2]) - radius


# define goal function g(x) >= 0
# where g(x) is the distance from the goal

@jax.jit
def goal_constraint(state, goal, radius):
    return jnp.linalg.norm(state[:2] - goal[:2]) - radius


# function to simulate the discrete time dynamics given initial state and control sequence
@functools.partial(jax.jit, static_argnames=["dt_dynamics"])
def simulate_discrete_time_dynamics(dt_dynamics, state, controls, t0, dt):
    states = [state]
    t = t0
    for c in controls:

        state = dt_dynamics(state, c, t)
        states.append(state)
        t += dt
    return jnp.stack(states)

# function to simulate the discrete time dynamics given initial state and control sequence
# function slightly modified to add noise 
@functools.partial(jax.jit, static_argnames=["dt_dynamics"])
def simulate_discrete_time_dynamics_with_noise(dt_dynamics, state, controls, t0, dt, noises):
    states = [state]
    t = t0
    for (c,noise) in zip(controls, noises):
        state = dt_robot_dynamics(state, c, t)#  + noise * dt # take out noises for now
        states.append(state)
        t += dt
    return jnp.stack(states, -1)

# jit the linearize constraint functions to make it run faster
linearize_obstacle = jax.jit(jax.vmap(jax.grad(obstacle_constraint), in_axes=[0, None, None]))
linearize_goal = jax.jit(jax.vmap(jax.grad(goal_constraint), in_axes=[0, None, None]))




In [734]:
# Polygon Helper Functions # 

@jax.jit
def polygon_constraint(state, polygon_vertices, buffer=1.0):
    """
    Returns the most violated constraint value for a convex polygon.
    Negative values mean the point is inside the polygon (safe).
    
    state: shape (2,) or (..., 2) — position of the robot
    polygon_vertices: shape (N, 2) — ordered CCW vertices of convex polygon
    buffer: float — buffer distance to expand polygon inward
    """
    # Compute edge vectors
    edges = jnp.roll(polygon_vertices, -1, axis=0) - polygon_vertices

    # Compute outward normals (rotate 90° clockwise)
    normals = jnp.stack([edges[:, 1], -edges[:, 0]], axis=-1)
    normals = normals / jnp.linalg.norm(normals, axis=1, keepdims=True)

    # Get offsets for each constraint: n · x ≤ n · p (where p is polygon vertex)
    offsets = jnp.sum(normals * polygon_vertices, axis=1) - buffer

    # For each constraint, compute n · x - b ≤ 0
    dot_products = jnp.dot(normals, state[:2])  # shape (N,)
    constraint_values = dot_products - offsets  # shape (N,)

    # Return the max violation (positive if outside any edge)
    return jnp.max(constraint_values)

def grad_polygon_constraint(state, polygon_vertices, buffer=0.0):
    # Compute all constraint values
    edges = jnp.roll(polygon_vertices, -1, axis=0) - polygon_vertices
    normals = jnp.stack([edges[:, 1], -edges[:, 0]], axis=-1)
    normals = normals / jnp.linalg.norm(normals, axis=1, keepdims=True)
    offsets = jnp.sum(normals * polygon_vertices, axis=1) - buffer

    dot_products = jnp.dot(normals, state[:2])
    constraint_values = dot_products - offsets

    # Find the index of the max violation edge
    idx = jnp.argmax(constraint_values)

    # Gradient is the normal of that edge
    grad = normals[idx]

    # If your state has more dims than 2, pad zeros for the other dims
    # For example, if state_dim > 2:
    grad_full = jnp.zeros_like(state)
    grad_full = grad_full.at[:2].set(grad)
    return grad_full



linearize_polygon = jax.jit(jax.vmap(jax.grad(polygon_constraint), in_axes=[0, None, None]))

polygon = jnp.array([
    [0.5, 1.0],
    [3.0, 1.0],
    [3.0, 3.0],
    [2.5, 3.0]

])

#robot_pos = jnp.array([1.5, 3])
#val = polygon_constraint(robot_pos, polygon)
#print("Constraint value:", val)  # Expect a negative value (inside)







In [735]:
# set up cvxpy problem variables and parameters
xs = cp.Variable([planning_horizon+1, state_dim])  # cvx variable for states
us = cp.Variable([planning_horizon, control_dim])  # cvx variable for controls
slack = cp.Variable(1) # slack variable to make sure the problem is feasible
As = [cp.Parameter([state_dim, state_dim]) for _ in range(planning_horizon)]  # parameters for linearized dynamics
Bs = [cp.Parameter([state_dim, control_dim]) for _ in range(planning_horizon)] # parameters for linearized dynamics
Cs = [cp.Parameter([state_dim]) for _ in range(planning_horizon)] # parameters for linearized dynamics

Gs = [cp.Parameter([state_dim]) for _ in range(planning_horizon+1)] # parameters for linearized constraints
hs = [cp.Parameter(1) for _ in range(planning_horizon+1)] # parameters for linearized constraints
Gs2 = [cp.Parameter([state_dim]) for _ in range(planning_horizon+1)] # parameters for linearized constraints
hs2 = [cp.Parameter(1) for _ in range(planning_horizon+1)] # parameters for linearized constraints

Goal_Gs = [cp.Parameter([state_dim]) for _ in range(planning_horizon+1)] # parameters for linearized constraints
Goal_hs = [cp.Parameter(1) for _ in range(planning_horizon+1)] # parameters for linearized constraints

Gs_polygon = [cp.Parameter([state_dim]) for _ in range(planning_horizon+1)] # parameters for linearized constraints
hs_polygon = [cp.Parameter(1) for _ in range(planning_horizon+1)] # parameters for linearized constraints


xs_previous = cp.Parameter([planning_horizon+1, state_dim]) # parameter for previous solution
us_previous = cp.Parameter([planning_horizon, control_dim]) # parameter for previous solution
initial_state = cp.Parameter([state_dim]) # parameter for current robot state

In [736]:
# set up cvxpy problem cost and constraints
beta1 = 0.2 # coefficient for control effort
beta2 = 5. # coefficient for progress
beta3 = 10. # coefficient for trust region
slack_penalty = 100. # coefficient for slack variable
markup = 1.0
goal_cost_weight = 1.
polygon_slack_penalty = 100.


## removed objective beta2 * (xs[-1,2]**2 + xs[-1,1]**2 - xs[-1,0])
objective = beta3 * (cp.sum_squares(xs - xs_previous) + cp.sum_squares(us - us_previous)) + slack_penalty * slack**2
constraints = [xs[0] == initial_state, slack >= 0] # initial state and slack constraint
for t in range(planning_horizon):

    objective += (beta1 * cp.sum_squares(us[t]) ) * markup**t # only penalizes control effort
    objective += goal_cost_weight * (cp.sum_squares((xs[t, :2] - goal_location)))*(1 + t/num_time_steps)  #.5 exponent, farther away, goal weight not as high
    objective += polygon_slack_penalty * cp.square(slack)  # or cp.power(slack, 3) for stronger penalty

    constraints += [xs[t+1] == As[t] @ xs[t] + Bs[t] @ us[t] + Cs[t]] # dynamics constraint
    constraints += [xs[t,-1] <= v_max, xs[t,-1] >= v_min, us[t,1] <= acceleration_max, us[t,1] >= acceleration_min, us[t,0] <= steering_max, us[t,0] >= steering_min] # control and velocity limit constraints
    constraints += [Gs[t] @ xs[t] + hs[t] >= -slack, Gs2[t] @ xs[t] + hs2[t] >= -slack] # linearized collision avoidance constraint for obstacles
    constraints += [Gs_polygon[t] @ xs[t] <= hs_polygon[t] + slack]

constraints += [xs[planning_horizon,-1] <= v_max, xs[planning_horizon,-1] >= v_min, Gs[planning_horizon] @ xs[planning_horizon] + hs[planning_horizon] >= -slack, Gs2[planning_horizon] @ xs[planning_horizon] + hs2[planning_horizon] >= -slack, Gs_polygon[planning_horizon] @ xs[planning_horizon] + hs_polygon[planning_horizon] >= -polygon_slack_penalty*slack] # constraints for last planning horizon step

prob = cp.Problem(cp.Minimize(objective), constraints) # construct problem
  

In [737]:
# initial states
robot_state = jnp.array([-1.5, -0.5, 0., 1.])  # robot starting state
robot_trajectory = [robot_state] # list to collect robot's state as it replans
sqp_list = [] # list to collect each sqp iteration 
robot_control_list = []  # list to collect robot's constrols as it replans
robot_trajectory_list = [] # list to collect robot's planned trajectories

# initial robot planned state and controls
previous_controls = jnp.zeros([planning_horizon, control_dim]) # initial guess for robot controls
previous_states =  simulate_discrete_time_dynamics(dt_robot_dynamics, robot_state, previous_controls, 0., dt) # initial guess for robot states
xs_previous.value = np.array(previous_states) # set xs_previous parameter value
us_previous.value = np.array(previous_controls) # set us_previous parameter value 

### (b) Replan at each timestep!
Hopefully you saw from above the need to replan at each time step.
Run the following cells and answer the question at the end.

(Just run the cell below)


In [738]:
# initial states
robot_state = jnp.array([-1.5, -0.5, 0., 1.])  # robot starting state
robot_trajectory = [robot_state] # list to collect robot's state as it replans
sqp_list = [] # list to collect each sqp iteration 
robot_control_list = []  # list to collect robot's constrols as it replans
robot_trajectory_list = [] # list to collect robot's planned trajectories

# initial robot planned state and controls
previous_controls = jnp.zeros([planning_horizon, control_dim]) # initial guess for robot controls
previous_states =  simulate_discrete_time_dynamics(dt_robot_dynamics, robot_state, previous_controls, 0., dt) # initial guess for robot states
xs_previous.value = np.array(previous_states) # set xs_previous parameter value
us_previous.value = np.array(previous_controls) # set us_previous parameter value 

# precompute the noise
key = jax.random.PRNGKey(0)
noise_covar = jnp.diag(jnp.array([0.1, 0.1, 0.05, 0.2])) # noise covariance
noises = jax.random.multivariate_normal(key, jnp.zeros(robot_state.shape), noise_covar, shape=(num_time_steps,))

Rerun the simulation, replanning at each time step, and adding some noise when computing the robot's next state.

(Just run the cell below)


In [ ]:
solver = cp.CLARABEL

for t in range(num_time_steps):
    initial_state.value = np.array(robot_state)
    sqp_solutions = [previous_states]
    

    
    for i in range(num_sqp_iterations):
        As_value, Bs_value, Cs_value = jax.vmap(linearize, in_axes=[None, 0, 0, None])(dt_robot_dynamics, previous_states[:-1], previous_controls, 0.)
        Gs_value = linearize_obstacle(previous_states, obstacle_location, obstacle_radius + robot_radius) 
        hs_value = jax.vmap(obstacle_constraint, [0, None, None])(previous_states, obstacle_location, obstacle_radius + robot_radius) - jax.vmap(jnp.dot, [0, 0])(Gs_value, previous_states)
        Gs2_value = linearize_obstacle(previous_states, obstacle_location2, obstacle_radius + robot_radius) 
        hs2_value = jax.vmap(obstacle_constraint, [0, None, None])(previous_states, obstacle_location2, obstacle_radius + robot_radius) - jax.vmap(jnp.dot, [0, 0])(Gs2_value, previous_states)
        Goal_Gs_value = linearize_goal(previous_states, goal_location, goal_radius + robot_radius) 
        Goal_hs_value = jax.vmap(goal_constraint, [0, None, None])(previous_states, goal_location, goal_radius + robot_radius) - jax.vmap(jnp.dot, [0, 0])(Goal_Gs_value, previous_states)
        
        Gs_polygon_value = jax.vmap(lambda s: grad_polygon_constraint(s, polygon, robot_radius))(previous_states)
        hs_polygon_value = jax.vmap(lambda s: polygon_constraint(s, polygon, robot_radius))(previous_states) - jax.vmap(jnp.dot, [0, 0])(Gs_polygon_value, previous_states)
        #Gs_crater_value, hs_crater_value = vmap_polygon_constraint(previous_states, polygon_crater, robot_radius)
        # Just call the function without vmap or jit

        




        for i in range(planning_horizon):
            As[i].value = np.array(As_value[i])
            Bs[i].value = np.array(Bs_value[i])
            Cs[i].value = np.array(Cs_value[i])
            Gs[i].value = np.array(Gs_value[i])
            hs[i].value = np.array(hs_value[i:i+1])
            Gs2[i].value = np.array(Gs2_value[i])
            hs2[i].value = np.array(hs2_value[i:i+1])
            Goal_Gs[i].value = np.array(Goal_Gs_value[i])
            Goal_hs[i].value = np.array(Goal_hs_value[i:i+1])

            Gs_polygon[i].value = np.array(Gs_polygon_value[i])          # shape: (state_dim,)
            hs_polygon[i].value = np.array(hs_polygon_value[i:i+1])      # shape: (1,)

        Gs[planning_horizon].value = np.array(Gs_value[planning_horizon])
        hs[planning_horizon].value = np.array(hs_value[planning_horizon:planning_horizon+1])
        Gs2[planning_horizon].value = np.array(Gs2_value[planning_horizon])
        hs2[planning_horizon].value = np.array(hs2_value[planning_horizon:planning_horizon+1])
        Goal_Gs[planning_horizon].value = np.array(Goal_Gs_value[planning_horizon])
        Goal_hs[planning_horizon].value = np.array(Goal_hs_value[planning_horizon:planning_horizon+1])

        Gs_polygon[planning_horizon].value = np.array(Gs_polygon_value[planning_horizon])
        hs_polygon[planning_horizon].value = np.array(hs_polygon_value[planning_horizon:planning_horizon+1])




        
        result = prob.solve(solver=solver)



        
        if us.value is None:
            print("No solution found")
            break
        previous_controls = us.value
        previous_states =  simulate_discrete_time_dynamics(dt_robot_dynamics, robot_state, previous_controls, 0., dt)
        sqp_solutions.append(previous_states)
        xs_previous.value = np.array(previous_states)
        us_previous.value = np.array(previous_controls)
    sqp_list.append(np.stack(sqp_solutions))
    robot_control = previous_controls[0]
    robot_control_list.append(robot_control)
    
    # get the robot next state using the control input
    robot_state = dt_robot_dynamics(robot_state, robot_control, 0.)#  + noises[t] * dt # take out noises for now
    
    # clipping the robot velocity so the problem doesn't become infeasible at the next step
    robot_state = robot_state.at[3].set(jnp.clip(robot_state[3], v_min, v_max)) 
    
    # add robot state and trajectory to the list
    robot_trajectory.append(robot_state)
    robot_trajectory_list.append(previous_states)
    
    # update the previous states and controls for the next iteration
    previous_states =  simulate_discrete_time_dynamics(dt_robot_dynamics, robot_state, previous_controls, 0., dt)
    
    
robot_trajectory = jnp.stack(robot_trajectory)
robot_controls = jnp.stack(robot_control_list)

ValueError: Incompatible dimensions (1, 4) (2, 1)

Visualize the results!

(Just run the cell below)


In [ ]:
def plot_polygon_obstacle(polygon, ax):
    polygon_loop = jnp.vstack([polygon, polygon[0]])
    ax.plot(polygon_loop[:, 0], polygon_loop[:, 1], 'k-')
    ax.fill(polygon_loop[:, 0], polygon_loop[:, 1], color='gray', alpha=0.5)

# plotting the results. No need to add comments here. Just run this cell to visualize the results
@interact(i=(0,num_time_steps-1), j=(0,num_sqp_iterations-1))



def plot(i, j):
    fig, axs = plt.subplots(1, 2, figsize=(12, 4), gridspec_kw={'width_ratios': [2, 1]})
    # fig, axs = plt.subplots(1,2, figsize=(10, 4))
    ax = axs[0]
    robot_position = robot_trajectory[i, :2]
    print("test",obstacle_location)
    circle1 = plt.Circle(robot_position, robot_radius, color='C0', alpha=0.4)
    circle2 = plt.Circle(obstacle_location, obstacle_radius, color='C1', alpha=0.4)
    circle3 = plt.Circle(obstacle_location2, obstacle_radius, color='C1', alpha=0.4)
    goal_loc = plt.Circle(goal_location, goal_radius, color='C2', alpha=0.6)
    ax.add_patch(circle1)
    ax.add_patch(circle2)
    ax.add_patch(circle3)
    ax.add_patch(goal_loc)
    plot_polygon_obstacle(polygon, ax)
    ax.plot(robot_trajectory[:,0], robot_trajectory[:,1], "o-", markersize=3, color='black')
    ax.plot(robot_trajectory_list[i][:,0], robot_trajectory_list[i][:,1], "o-", markersize=3, color='red', label="Planned")
    # Plot planned trajectory for the selected SQP iteration
    planned_trajectory = sqp_list[i][j]
    ax.plot(planned_trajectory[:, 0], planned_trajectory[:, 1], "o-", markersize=3, color='green', alpha=0.4, label="SQP iteration %d" % j)
    ax.scatter(robot_trajectory[i:i+1,0], robot_trajectory[i:i+1,1], s=30,  color='C0', label="Robot")
    ax.set_xlim([-2, 7])
    ax.grid()
    ax.legend()
    ax.axis("equal")

    ax.set_title("heading=%.2f velocity=%.2f"%(robot_trajectory[i,2], robot_trajectory[i,3]))
    
    ax = axs[1]
    ax.plot(robot_controls)
    ax.scatter([i], robot_controls[i:i+1, 0], label="$tan(\\delta)$", color='C0')
    ax.scatter([i], robot_controls[i:i+1, 1], label="Acceleration", color='C1')

    ax.hlines(steering_min, 0, num_time_steps-1, color='C0', linestyle='--')
    ax.hlines(steering_max, 0, num_time_steps-1, color='C0', linestyle='--')
    ax.hlines(acceleration_min, 0, num_time_steps-1, color='C1', linestyle='--')
    ax.hlines(acceleration_max, 0, num_time_steps-1, color='C1', linestyle='--')
    
    ax.plot(robot_trajectory[:,-1], markersize=3, color='C2')
    ax.scatter([i], robot_trajectory[i:i+1, 3], label="Velocity", color='C2')
    ax.hlines(v_min, 0, num_time_steps-1, color='C2', linestyle='--')
    ax.hlines(v_max, 0, num_time_steps-1, color='C2', linestyle='--')
    ax.set_xlim([0, num_time_steps])
    ax.set_ylim([-2, 2])
    ax.set_xlabel("Time step")
    ax.set_ylabel("Control")
    ax.set_title("Velocity, steering and acceleration")
    ax.legend()
    ax.grid()

interactive(children=(IntSlider(value=24, description='i', max=49), IntSlider(value=4, description='j', max=9)…

Is it guaranteed that your system will not hit the obstacle if we just apply MPC in the way we did above?
If not, what are some techniques you could try to reduce the risk of colliding into the obstacles?

(Optional) What if the obstacles were not stationary. But rather, they were moving. How would the problem change if the obstacle's motion were fully known (e.g., we knew exactly where the obstacle would be at any point in time), or if the obstacle's motion was uncertain (e.g., the obstacles were pedestrians, or space debris whose motion is not fully known). What are some techniques that could be applied to reduce the risk of collision?

It is not guaranteed the system will not hit the obstacle. In my code, even with MPC applied, the first obstacle appears hit. In this case it was hit due to a sharp change in direction across only one timepoint. This indicates more safety measures should be put in place. Examples of those include increasing the size of the obstacle (similar to CBF applications) to compensate for error. While much more computationally intensive, the timestep frequency could be increased, which would make the magnitude of noise at each timestep smaller and give more opportunities for trajectory correction. Control restrictions can be put on the velocity to restrict movement, which also would give more opportunities for course correction. 

If the obstacles were moving but the motion was known, that should be able to be factored into the state and the problem should not change much. If the obstacle's motion was uncertain, then the planning horizon would need to be significantly decreased. A long planning horizon is useless if the states change unpredictably, and the desired trajectory may need to change on a whim. Adjusting the controls to allow for more leeway in sharp changes in acceleration would be helpful in dodging swift obstacles moving in the direction of the robot or its trajectory.